In [3]:
from flask import Flask, render_template, request, session, url_for, redirect, jsonify,make_response
import pymysql
from pyresparser import ResumeParser
from werkzeug.utils import secure_filename
from models.keras_first_go import KerasFirstGoModel
from clear_bash import clear_bash
import os

from FacebookPostsScraper import FacebookPostsScraper as Fps
from pprint import pprint as pp


app = Flask(__name__)
app.secret_key = 'random string'
cleaner=clear_bash()

#app.config[‘UPLOAD_FOLDER’]=
app.config['UPLOADED_FILE'] = 'static/ResumeFiles'



#train_model()
#processed_text = first_go_model.prediction("Oracle Soap Sdlc C Engineering Opencv Architecture Android Sql Java Html Database Agile Technical")
#result = {'Job': processed_text}
#print(result)

def getrecommendedjob(skills):
    first_go_model = KerasFirstGoModel()
    print(skills)
    processed_text = first_go_model.prediction(skills)    
    result = {'Job': processed_text}
    print(result)
    return processed_text
     
     
#Database Connection
def dbConnection():
    connection = pymysql.connect(host="localhost", user="root", password="root", database="resumeverification")
    return connection


#close DB connection
def dbClose():
    dbConnection().close()
    return


@app.route('/index')
@app.route('/')
def index():
    return render_template('index.html')

#logout code
@app.route('/logout')
def logout():
    session.pop('user')
    return redirect(url_for('index'))


#login code
@app.route('/login', methods=["GET","POST"])
def login():
    msg = ''
    if request.method == "POST":
        session.pop('user',None)
        mailid = request.form.get("mailid")
        password = request.form.get("pas")
        #print(mobno+password)
        con = dbConnection()
        cursor = con.cursor()
        result_count = cursor.execute('SELECT * FROM userdetails WHERE emailid = %s AND password = %s', (mailid, password))
        #a= 'SELECT * FROM userdetails WHERE mobile ='+mobno+'  AND password = '+ password
        #print(a)
        #result_count=cursor.execute(a)
        # result = cursor.fetchone()
        if result_count>0:
            print(result_count)
            session['user'] = mailid
            return redirect(url_for('home'))
        else:
            print(result_count)
            msg = 'Incorrect username/password!'
            return msg
        #dbClose()
    return redirect(url_for('index'))


#user register code
@app.route('/userRegister', methods=["GET","POST"])
def userRegister():
    if request.method == "POST":
        try:
            status=""
            name = request.form.get("name")
            address = request.form.get("address")
            mailid = request.form.get("mailid")
            mobile = request.form.get("mobile")
            pass1 = request.form.get("pass1")
            con = dbConnection()
            cursor = con.cursor()
            cursor.execute('SELECT * FROM userdetails WHERE emailid = %s', (mailid))
            res = cursor.fetchone()
            #res = 0
            if not res:
                sql = "INSERT INTO userdetails (name, address, emailid, mobileno, password) VALUES (%s, %s, %s, %s, %s)"
                val = (name, address, mailid, mobile, pass1)
                print(sql," ",val)
                cursor.execute(sql, val)
                con.commit()
                status= "success"
                return redirect(url_for('index'))
            else:
                status = "Already available"
            #return status
            return redirect(url_for('index'))
        except:
            print("Exception occured at user registration")
            return redirect(url_for('index'))
        finally:
            dbClose()
    return redirect(url_for('index'))


@app.route('/home')
def home():
    if 'user' in session:

        return render_template('home.html', user=session['user'])
    return redirect(url_for('index'))

#import os

@app.route('/uploader', methods = ['GET', 'POST'])
def upload_file():
    if 'user' in session:
        if request.method == 'POST':
            f = request.files['file']
            print(f)
            filename = secure_filename(f.filename)
            f.save(os.path.join(app.config['UPLOADED_FILE'], filename))

            filename = os.path.abspath(app.config['UPLOADED_FILE']+"//"+filename)#os.path.abspath(filename)
            
            filename = filename#'ningesh.pdf'
 #           print("=======")
 #           print(filename)
 #           print("=======")
            listofcontent = ['pdf', 'docx']
            splitfilename = filename.split(".")
            if splitfilename[1] in listofcontent:
                data = ResumeParser(filename).get_extracted_data()
                print(data)
                #print(type(['Soap', 'Agile', 'Sdlc', 'Java', 'Sql', 'C', 'Android', 'Engineering', 'Html', 'Architecture', 'Opencv', 'Database', 'Oracle', 'Technical']))
                                
                skillset=data['skills']
                print(skillset)
                dataop=''
                for ik in skillset:
                    dataop=dataop+ik+" "
                
                print('=======')
                print(dataop)
                #global first_go_model
                #processed_text1 = first_go_model.prediction("Oracle Soap Sdlc C Engineering Opencv Architecture Android Sql Java Html Database Agile Technical")
                #processed_text1 = first_go_model.prediction(dataop)
                
                processed_text1 = getrecommendedjob(dataop)
                result1 = {'Job type suited for this ': processed_text1}
                #result1 = {'Job type suited for this ': processed_text1}
                print(result1)
                
                return render_template('recommend.html', user=session['user'], data=data, job=processed_text1)
            else:
                print('Not able to scan data please provide pdf format')


            #return 'file uploaded successfully'
        return render_template('home.html', user=session['user'])
    return redirect(url_for('index'))


@app.route('/FBLinkuploader', methods = ['GET', 'POST'])
def FBLinkuploader():
    if 'user' in session:
        if request.method == 'POST':
            fblink = request.form.get("fblink")
            fbposts = FB_post_fetch(fblink)           
            
            return render_template('FetchedPost.html', data=fbposts, user=session['user'])
        return render_template('home.html', user=session['user'])
    return redirect(url_for('index'))


def FB_post_fetch(link):
    # Enter your Facebook email and password
    email = 'YOUR_EMAIL'
    password = 'YOUR_PASSWORD'
    # Instantiate an object
    fps = Fps(email, password, post_url_text='Full Story')
    # Example with single profile
    #single_profile = 'https://www.facebook.com/BillGates'
    data = fps.get_posts_from_profile(link)
    pp(data)
    return data


if __name__ == '__main__':
    #app.run(debug="True")
    app.run('0.0.0.0')

Using TensorFlow backend.
C:\Users\ningesh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ningesh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ningesh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ningesh\App

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Oct/2020 14:56:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2020 14:56:02] "GET /static/css/bootstrap.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2020 14:56:02] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2020 14:56:02] "GET /static/css/fontawesome-all.min.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2020 14:56:02] "GET /static/css/owl.carousel.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2020 14:56:02] "GET /static/css/owl.theme.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2020 14:56:02] "GET /static/css/popup-box.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2020 14:56:02] "GET /static/js/jquery-2.2.3.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2020 14:56:02] "GET /static/js/jquery.magnific-popup.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2020 14:56:03] "GET /static/js/move-top.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2020 14:56:03] "GET /static/js/easing.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2020

1


127.0.0.1 - - [29/Oct/2020 14:56:44] "GET /home HTTP/1.1" 200 -


...

[{'description': 'Understanding this concept will help the world make the most '
                 'of its efforts and funding as we work together to avoid a '
                 'climate disaster.',
  'external_links': [],
  'images': [],
  'like_url': '',
  'post_url': '',
  'published': '२९ सप्टेंबर रोजी ४:०६ PM वाजता'},
 {'description': 'What the world does in the next few months is going to make '
                 'a big difference. Read our 2020 Goalkeepers Report to learn '
                 'what it will take to end the pandemic.',
  'external_links': [],
  'images': [],
  'like_url': '',
  'post_url': '',
  'published': '२४ सप्टेंबर रोजी २:३६ PM वाजता'},
 {'description': 'Our foundation is partnering with DonorsChoose to support '
                 'teachers as they get ready to go back to school—in whatever '
                 'form that takes—and will boost your donations to the '
                 'classroom project of your choice today.',
  'external_links': [],
  'images': [],
 

127.0.0.1 - - [29/Oct/2020 15:14:15] "POST /FBLinkuploader HTTP/1.1" 200 -


[]


127.0.0.1 - - [29/Oct/2020 15:15:03] "POST /FBLinkuploader HTTP/1.1" 200 -


[{'description': 'Understanding this concept will help the world make the most '
                 'of its efforts and funding as we work together to avoid a '
                 'climate disaster.',
  'external_links': [],
  'images': [],
  'like_url': '',
  'post_url': '',
  'published': '२९ सप्टेंबर रोजी ४:०६ PM वाजता'},
 {'description': 'What the world does in the next few months is going to make '
                 'a big difference. Read our 2020 Goalkeepers Report to learn '
                 'what it will take to end the pandemic.',
  'external_links': [],
  'images': [],
  'like_url': '',
  'post_url': '',
  'published': '२४ सप्टेंबर रोजी २:३६ PM वाजता'},
 {'description': 'Our foundation is partnering with DonorsChoose to support '
                 'teachers as they get ready to go back to school—in whatever '
                 'form that takes—and will boost your donations to the '
                 'classroom project of your choice today.',
  'external_links': [],
  'images': [],
 